In [28]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Параметры сетки и времени
Nx = 50  # Число узлов по оси x
Ny = 50  # Число узлов по оси y
Lx = 1   # Длина области по x
Ly = 1   # Длина области по y
hx = Lx / Nx  # Шаг по x
hy = Ly / Ny  # Шаг по y
T = 1.0       # Конечное время
dt = 0.001    # Шаг по времени
Nt = int(T / dt)  # Число временных шагов

x = np.linspace(0, Lx, Nx + 1)
y = np.linspace(0, Ly, Ny + 1)
X, Y = np.meshgrid(x, y)

# Начальные условия
u = np.zeros((Nx + 1, Ny + 1, Nt + 1))  # Массив решения
u[:, :, 0] = 0  # u(x, y, 0) = 0

# Метод прогонки для одномерного уравнения
def tdma(a, b, c, d):
    n = len(d)
    w = np.zeros(n - 1)
    g = np.zeros(n)
    p = np.zeros(n)
    
    # Прямой ход
    w[0] = c[0] / b[0]
    g[0] = d[0] / b[0]
    for i in range(1, n - 1):
        w[i] = c[i] / (b[i] - a[i - 1] * w[i - 1])
    for i in range(1, n):
        g[i] = (d[i] - a[i - 1] * g[i - 1]) / (b[i] - a[i - 1] * w[i - 1])
    
    # Обратный ход
    p[n - 1] = g[n - 1]
    for i in range(n - 2, -1, -1):
        p[i] = g[i] - w[i] * p[i + 1]
    
    return p

# Решение уравнения методом переменных направлений
for k in range(1, Nt + 1):
    # Шаг 1: Прогонка по x
    for j in range(1, Ny):
        a = np.full(Nx - 1, -dt / hx**2)
        b = np.full(Nx, 1 + 2 * dt / hx**2)
        c = np.full(Nx - 1, -dt / hx**2)
        d = u[1:Nx, j, k - 1] + dt * np.sin(np.pi * x[1:Nx]) * y[j] * (y[j] - 1) * (k * dt)
        u[1:Nx, j, k] = tdma(a, b, c, d)

    # Граничные условия по x
    u[0, :, k] = 0
    u[Nx, :, k] = 0

    # Шаг 2: Прогонка по y
    for i in range(1, Nx):
        a = np.full(Ny - 1, -dt / hy**2)
        b = np.full(Ny, 1 + 2 * dt / hy**2)
        c = np.full(Ny - 1, -dt / hy**2)
        d = u[i, 1:Ny, k] + dt * np.sin(np.pi * x[i]) * y[1:Ny] * (y[1:Ny] - 1) * (k * dt)
        u[i, 1:Ny, k] = tdma(a, b, c, d)
    
    # Граничные условия по y
    u[:, 0, k] = 0
    u[:, Ny, k] = 0

# Визуализация результата
def plot_solution(t_index):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(X, Y, u[:, :, t_index], cmap='viridis', edgecolor='none')
    
    ax.set_title(f'Solution at t = {t_index * dt:.3f}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('U')
    
    # Настройка отображения чисел на осях
    ax.set_xticks(np.linspace(0, Lx, 5))
    ax.set_yticks(np.linspace(0, Ly, 5))
    ax.set_zticks(np.linspace(np.min(u), np.max(u), 5))
    
    plt.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
    plt.show()

# Интерактивный виджет для выбора времени
interactive_plot = interactive(plot_solution, t_index=(0, Nt, 1))
interactive_plot


interactive(children=(IntSlider(value=500, description='t_index', max=1000), Output()), _dom_classes=('widget-…